In [114]:
# libraries
import datetime
import time
import json
from pymongo import MongoClient
from faker import Faker

# Data generation

In [115]:
class Lab2models:
    def data_generator(self, num_companies, people_per_company):
        # Connect to MongoDB - Note: added my local host
        client = MongoClient('mongodb://localhost:27017/')
        db = client['lab2']
        # delete collection data if exists from all fo the 3 models
        db.drop_collection("m1_people")
        db.drop_collection("m1_companies")
        db.drop_collection("m2_people")
        db.drop_collection("m3_companies")
        # create and obtain collections
        m1_people = db.create_collection('m1_people')
        m1_companies = db.create_collection('m1_companies')
        m2_people = db.create_collection('m2_people')
        m3_companies = db.create_collection('m3_companies')
        # It will generate data in spanish: 'es_ES'
        fake = Faker(['es_ES'])
        # Generate company data
        company_data = []
        for i in range(num_companies):
            company_name = fake.company()
            domain = fake.domain_name()
            company = {
                "domain": domain,
                "email": f"info@{domain}",
                "name": company_name,
                "url": f"www.{domain}",
                "vatNumber": fake.bothify(text='??######')
            }
            company_data.append(company)
        
        # Insert companies for Model 1
        company_ids = m1_companies.insert_many(company_data).inserted_ids

        # Generate people's data
        person_data = []
        for i in range(num_companies * people_per_company):
            company_idx = i % num_companies  # Distribute the people among companies
            
            if fake.random_element(elements=(True, False)):
                first_name = fake.first_name_male()
                sex = 'M'
            else:
                first_name = fake.first_name_female()
                sex = 'F'
                
            last_name = fake.last_name()
            full_name = f"{first_name} {last_name}"
            birth_date = fake.date_of_birth(minimum_age=20, maximum_age=65)
            birth_datetime = datetime.datetime.combine(birth_date, datetime.time())
            age = (datetime.datetime.now().date() - birth_date).days // 365
            
            username_full = f"{first_name.lower()}.{last_name.lower()}".replace(" ", "")
            username_short = f"{first_name.lower()[0]}{last_name.lower()}".replace(" ", "")
            
            person = {
                "age": age,
                "companyEmail": f"{username_full}@{company_data[company_idx]['domain']}",
                "dateOfBirth": birth_datetime,
                "email": f"{username_short}@{fake.free_email_domain()}",
                "firstName": first_name,
                "fullName": full_name,
                "sex": sex 
            }
            person_data.append((person, company_idx))

        # MODEL 1
        for i, (person, company_idx) in enumerate(person_data):
            m1_person = person.copy()
            m1_person["worksIn"] = company_ids[company_idx]  # Reference to company
            m1_people.insert_one(m1_person)
        
        # MODEL 2
        for i, (person, company_idx) in enumerate(person_data):
            m2_person = person.copy()
            m2_person["worksIn"] = company_data[company_idx]  # Embed company document
            m2_people.insert_one(m2_person)
        
        # MODEL 3
        persons_by_company = {}
        for person, company_idx in person_data:
            if company_idx not in persons_by_company:
                persons_by_company[company_idx] = []
            persons_by_company[company_idx].append(person.copy())
        
        for i, company in enumerate(company_data):
            m3_company = company.copy()
            m3_company["staff"] = persons_by_company.get(i, [])
            m3_companies.insert_one(m3_company)
        
        return num_companies, people_per_company
        
    def close_connection(self):
        if hasattr(self, 'client'):
            self.client.close()

In [116]:
if __name__ == "__main__":
    lab = Lab2models()

    num_companies = 30       
    people_per_company = 100  
    
    print(f"Generating data for {num_companies} companies with approximately {people_per_company} people per company...")
    print(f"Total expected documents: {num_companies} companies and {num_companies * people_per_company} people")

Generating data for 30 companies with approximately 100 people per company...
Total expected documents: 30 companies and 3000 people


# Queries

In [117]:
client = MongoClient('mongodb://localhost:27017/')
db = client['lab2']

### Q1: For each person, retrieve their full name and their company’s name.

In [118]:
def run_query(db, collection, pipeline, model_name):
    print(f"\n# {model_name}")
    start_time = time.time()
    results = list(db[collection].aggregate(pipeline))
    
    # show the first 3 results
    for i, result in enumerate(results[:3]):
        print(result)
    if len(results) > 3:
        print(f"... and {len(results) - 3} other results")
    
    generation_time = time.time() - start_time
    print(f"Time taken: {generation_time:.4f} seconds")
    
    return generation_time

In [119]:
pipeline_1 = [{"$lookup": {"from": "m1_companies", "localField": "worksIn", "foreignField": "_id", "as": "companyName"}},
    {"$unwind": "$companyName"}, {"$project": {"_id": 0, "fullName": 1, "companyName": "$companyName.name"}}]

pipeline_2 = [{"$project": {"_id": 0, "fullName": 1, "companyName": "$worksIn.name"}}]

pipeline_3 = [{"$unwind": "$staff"},{"$project": {"_id": 0, "fullName": "$staff.fullName", "companyName": "$name"}}]

In [120]:
time1 = run_query(db, "m1_people", pipeline_1, "Model 1: Two types of documents")
time2 = run_query(db, "m2_people", pipeline_2, "Model 2: One document for person with embedded company")
time3 = run_query(db, "m3_companies", pipeline_3, "Model 3: One document for company with embedded people")


# Model 1: Two types of documents
{'fullName': 'Florencia Medina', 'companyName': 'Hnos Lara S.L.L.'}
{'fullName': 'Ceferino Coca', 'companyName': 'Logística del Norte S.L.N.E'}
{'fullName': 'Lourdes Piñeiro', 'companyName': 'Distribuciones Soler & Asociados S.Coop.'}
... and 2997 other results
Time taken: 0.0750 seconds

# Model 2: One document for person with embedded company
{'fullName': 'Florencia Medina', 'companyName': 'Hnos Lara S.L.L.'}
{'fullName': 'Ceferino Coca', 'companyName': 'Logística del Norte S.L.N.E'}
{'fullName': 'Lourdes Piñeiro', 'companyName': 'Distribuciones Soler & Asociados S.Coop.'}
... and 2997 other results
Time taken: 0.0074 seconds

# Model 3: One document for company with embedded people
{'fullName': 'Florencia Medina', 'companyName': 'Hnos Lara S.L.L.'}
{'fullName': 'Florentino Robledo', 'companyName': 'Hnos Lara S.L.L.'}
{'fullName': 'Calixto Manrique', 'companyName': 'Hnos Lara S.L.L.'}
... and 2997 other results
Time taken: 0.0055 seconds


In [121]:
# Comparison 
print("\nModel comparison:")
print(f"Model 1: {time1:.4f}s, Model 2: {time2:.4f}s, Model 3: {time3:.4f}s")
print(f"The fastest model is: Model {1 if time1 < time2 and time1 < time3 else 2 if time2 < time3 else 3}")


Model comparison:
Model 1: 0.0750s, Model 2: 0.0074s, Model 3: 0.0055s
The fastest model is: Model 3
